# Fine-tune and Evaluate Amazon Nova Micro model provided by Amazon Bedrock: End-to-End

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Nova Micro](https://aws.amazon.com/bedrock/nova/) model in Bedrock. You can also do this through the Bedrock Console.

<div class="alert alert-block alert-warning">

<b>Warning:</b> This module cannot be executed in Workshop Studio Accounts, and you will have to run this notebook in your own account.

</div>

### A Summarization Use Case

In this notebook, we build an end-to-end workflow for fine-tuning and evaluating the Foundation Models (FMs) in Amazon Bedrock. We choose [Amazon Nova Micro](https://aws.amazon.com/bedrock/nova/) as our FM to perform the customization through fine-tuning, we then create on demand inference deployment of the fine-tuned model, and test the on demand invocation using various API methods.

> *This notebook should work well with the **`Data Science 3.0`**, **`Python 3`**, and **`ml.c5.2xlarge`** kernel in SageMaker Studio*

## Prerequisites

 - Make sure you have executed `01_setup.ipynb` notebook.
 - Make sure you are using the same kernel and instance as `01_setup.ipynb` notebook.

In this notebook we demonstrate using Boto3 sdk for the fine-tuning and provisioning of [Nova Micro](https://aws.amazon.com/bedrock/nova/) model in Bedrock. You can also do this through the Bedrock Console.

<div class="alert alert-block alert-warning">

<b>Warning:</b> This notebook will create provisioned throughput for testing the fine-tuned model. Therefore, please make sure to delete the provisioned throughput as mentioned in the last section of the notebook, otherwise you will be charged for it, even if you are not using it.

</div>

## Setup

Install and import all the needed libraries and dependencies to complete this notebook.

Please ignore error messages related to pip's dependency resolver.

In [ ]:
# # install the fmeval package for foundation model evaluation

!rm -Rf ~/.cache/pip/*
!pip install tokenizers==0.12.1
!pip install -qU fmeval==0.3.0
!pip install awscurl

#### Setup Tips:

⚠️ ⚠️ ⚠️ If you have trouble installing fmeval, please make sure you have the dependencies installed correctly. See full list of dependencies [here](https://github.com/aws/fmeval/blob/main/poetry.lock). ⚠️ ⚠️ ⚠️ 

In [ ]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Fetching varialbes from `00_setup.ipynb` notebook. 

In [ ]:
%store -r role_arn
%store -r s3_train_uri
%store -r s3_validation_uri
%store -r s3_test_uri
%store -r bucket_name

In [ ]:
import pprint
pprint.pp(role_arn)
pprint.pp(s3_train_uri)
pprint.pp(s3_validation_uri)
pprint.pp(s3_test_uri)
pprint.pp(bucket_name)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import json
import os
import sys
import boto3
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
session = boto3.session.Session()
region = "us-east-1" # Hardcoded to us-west-2 region
sts_client = boto3.client('sts')
s3_client = boto3.client('s3')
aws_account_id = sts_client.get_caller_identity()["Account"]
bedrock = boto3.client(service_name="bedrock", region_name=region, endpoint_url='https://preprod.us-east-1.controlplane.bedrock.aws.dev')
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region, endpoint_url='https://preprod.us-east-1.dataplane.bedrock.aws.dev/')

In [ ]:
test_file_name = "test-cnn-10.jsonl"
data_folder = "fine-tuning-datasets-nova"

## Create the Fine-Tuning Job

<div class="alert alert-block alert-info">

<b>Note:</b> Fine-tuning job will take around 60mins to complete with 5K records.</div>

Amazon Nova Micro customization hyperparameters: 

- `epochs`: The number of iterations through the entire training dataset and can take up any integer values in the range of 1-10, with a default value of 2.

- `batchSize`: The number of samples processed before updating model parametersand can take up any integer values in the range of 1-64, with a default value of 1.

- `learningRate`:	The rate at which model parameters are updated after each batch	which can take up a float value betweek 0.0-1.0 with a default value set to	1.00E-5.


For guidelines on setting hyper-parameters refer to the guidelines provided [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html)

In [ ]:
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# Choose the foundation model you want to customize and provide ModelId(find more about model reference at https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-reference.html)
base_model_id = "amazon.nova-micro-v1:0:128k"

# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
customization_type = "FINE_TUNING"

# Specify the roleArn for your customization job
customization_role = role_arn

# Create a customization job name
customization_job_name = f"amazon-nova-micro-custom-job-{ts}"

# Create a customized model name for your fine-tuned Nova Micro model
custom_model_name = f"amazon-nova-micro-custom-model-{ts}"

# Define the hyperparameters for fine-tuning Nova Micro model
hyper_parameters = {
        "epochCount": "2",
        "batchSize": "1",
        "learningRate": "0.00005",
    }

# Specify your data path for training, validation(optional) and output
training_data_config = {"s3Uri": s3_train_uri}

# # uncomment the below section if you have validation dataset and provide the s3 uri for it. 

validation_data_config = {
        "validators": [{
            "s3Uri": s3_validation_uri
        }]
    }

output_data_config = {"s3Uri": f's3://{bucket_name}/outputs/output-{custom_model_name}'}

# # Create the customization job
bedrock.create_model_customization_job(
    customizationType=customization_type,
    jobName=customization_job_name,
    customModelName=custom_model_name,
    roleArn=customization_role,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyper_parameters,
    trainingDataConfig=training_data_config,
    validationDataConfig=validation_data_config,
    outputDataConfig=output_data_config

)

## Check Customization Job Status

In [ ]:
import time
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
print(fine_tune_job)

while fine_tune_job == "InProgress":
    time.sleep(60)
    fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
    print (fine_tune_job)

## Retrieve Custom Model

Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned Nova Micro model.

In [ ]:
# You can list your custom models using the command below
bedrock.list_custom_models()

<div class="alert alert-block alert-info">

<b>Note:</b> Please make sure your customization job status is "completed" before proceeding to retrieve the modelArn, otherwise you will run into errors. </div>

In [ ]:
# retrieve the modelArn of the fine-tuned model
fine_tune_job = bedrock.get_custom_model(modelIdentifier=custom_model_name)
custom_model_id = fine_tune_job['modelArn']

In [ ]:
output_job_name = "model-customization-job-"+fine_tune_job['jobArn'].split('/')[-1]
output_job_name

## Visualize Training and Validation Loss

Now that we have completed fine-tuning job, lets visualize our results to see if our job is not underfitting or overfitting. 
Download model customization job metrics from S3 and plot the learning curves.

In [ ]:
output_metrics_path = f"fine-tuning-datasets-nova/{output_job_name}"

In [ ]:
!mkdir $output_metrics_path

In [ ]:
train_metrics_s3_prefix=f'outputs/output-{custom_model_name}/{output_job_name}/training_artifacts/step_wise_training_metrics.csv'
validation_metrics_s3_prefix=f'outputs/output-{custom_model_name}/{output_job_name}/validation_artifacts/post_fine_tuning_validation/validation/validation_metrics.csv'
train_metrics_name='train_metrics.csv'
validation_metrics_name='validation_metrics.csv'
train_file_name_local=output_metrics_path+'/'+train_metrics_name
validation_file_name_local=output_metrics_path+'/'+validation_metrics_name

In [ ]:
s3_client.download_file(bucket_name, train_metrics_s3_prefix, train_file_name_local)
s3_client.download_file(bucket_name, validation_metrics_s3_prefix, validation_file_name_local)

In [ ]:
train_data = pd.read_csv(train_file_name_local)
'''The training loss is at an iteration level. To calculate loss at the epoch level,

    average the iteration-level loss for each epoch'''
train_metrics_epoch=train_data.groupby('epoch_number').mean()
validation_metrics_epoch=pd.read_csv(validation_file_name_local)
plt.plot(validation_metrics_epoch.epoch_number, validation_metrics_epoch.validation_loss,label='validation')
plt.plot(train_metrics_epoch.index, train_metrics_epoch.training_loss,label='training')
plt.title('Training vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## Deploy the Fine-tuned Model

After fine-tuning is complete, we need to deploy the model to make it available for inference. The following command creates a deployment for our fine-tuned Nova Micro model.

In [ ]:
!awscurl -i --service bedrock -X POST  https://preprod.us-east-1.controlplane.bedrock.aws.dev/model-customization/custom-model-deployments --data '{"modelDeploymentName":"novafinetuned","modelArn":"arn:aws:bedrock:us-east-1:656681812069:custom-model/amazon.nova-micro-v1:0:128k/3q67l58zazx9","description":"foo","clientRequestToken":"foo","tags":[{"key":"test","value":"test"}]}'

## Testing the Deployed Model

Now that we have deployed our fine-tuned Nova Micro model, we can test it using various API methods provided by Amazon Bedrock. Each method offers different capabilities for interacting with the model.

### Converse API
The Converse API allows for synchronous conversation with the model, receiving the complete response at once.

In [ ]:
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1', endpoint_url='https://preprod.us-east-1.dataplane.bedrock.aws.dev/')

try:
    response = brt.converse(
        modelId='arn:aws:bedrock:us-east-1:656681812069:custom-model-deployment/uholep6rb2hc',
        messages=[
            {
                'role': 'user',
                'content': [
                    {
                        'text': 'tell me a joke in 100 words',
                    }
                ]
            }
        ]
    )

    print("Request ID:", response['ResponseMetadata']['RequestId'])
    result = response.get('output')
    print(result)
    print("\n\n\n")
    print(result['message']['content'][0]['text'])

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])      

### Converse Stream API
The Converse Stream API provides the response as a stream of tokens, allowing for faster time-to-first-token and progressive rendering of the response.

In [ ]:
import boto3
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1', endpoint_url='https://preprod.us-east-1.dataplane.bedrock.aws.dev/')

try:
    response = brt.converse_stream(
        
        modelId='arn:aws:bedrock:us-east-1:656681812069:custom-model-deployment/uholep6rb2hc',
        messages=[
            {
                'role': 'user',
                'content': [
                    {
                        'text': 'tell me a joke in 100 words',
                    }
                ]
            }
        ]
    )

    for chunk in response["stream"]:
        if "contentBlockDelta" in chunk:
            text = chunk["contentBlockDelta"]["delta"]["text"]
            print(text, end="")
    
except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])

### Invoke API
The Invoke API allows for direct model invocation with custom system prompts and inference parameters.

In [ ]:
import boto3
import json

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1', endpoint_url='https://preprod.us-east-1.dataplane.bedrock.aws.dev/')

# Define your system prompt(s).
system_list = [
            {
                "text": "Act as a creative writing assistant. When the user provides you with a topic, write a short story about that topic."
            }
]
# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": "A camping trip"}]}]

# Configure the inference parameters.
inf_params = {"maxTokens": 500, "topP": 0.9, "topK": 20, "temperature": 0.7}
request_body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params

}

body = json.dumps(request_body)
try:
    response = brt.invoke_model(
        modelId='arn:aws:bedrock:us-east-1:656681812069:custom-model-deployment/uholep6rb2hc',
        body=body

    )
    print("Request ID:", response['ResponseMetadata']['RequestId'])

    # Decode the response body.
    model_response = json.loads(response["body"].read())
    # Extract and print the response text.
    response_text = model_response["output"]["message"]["content"][0]["text"]
    print(response_text)

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])


### Invoke Stream API
Similar to the Converse Stream API, this provides streaming responses but with the Invoke API's flexibility for custom configurations.

In [ ]:
import boto3
import json
from datetime import datetime

brt = boto3.client(service_name='bedrock-runtime', region_name = 'us-east-1', endpoint_url='https://preprod.us-east-1.dataplane.bedrock.aws.dev//')

system_list = [
            {
                "text": "Act as a creative writing assistant. When the user provides you with a topic, write a short story about that topic."
            }
]
# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": "A camping trip"}]}]

# Configure the inference parameters.
inf_params = {"maxTokens": 500, "topP": 0.9, "topK": 20, "temperature": 0.7}

request_body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

try:
    start_time = datetime.now()
    chunk_count = 0
    time_to_first_token = None
    response = brt.invoke_model_with_response_stream(
        modelId='arn:aws:bedrock:us-east-1:656681812069:custom-model-deployment/uholep6rb2hc',
        body=json.dumps(request_body)
    )

    stream = response.get("body")
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                # Print the response chunk
                chunk_json = json.loads(chunk.get("bytes").decode())
                # Pretty print JSON
                # print(json.dumps(chunk_json, indent=2, ensure_ascii=False))

                content_block_delta = chunk_json.get("contentBlockDelta")
                if content_block_delta:
                    if time_to_first_token is None:
                        time_to_first_token = datetime.now() - start_time
                        print(f"Time to first token: {time_to_first_token}")
    
                    chunk_count += 1
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S:%f")
                    # print(f"{current_time} - ", end="")
                    print(content_block_delta.get("delta").get("text"), end="")

        print(f"Total chunks: {chunk_count}")

    else:
        print("No response stream received.")

except Exception as e:
    # Print the full error response for debugging
    print("Error:", e)
    # Extract and print the Request ID from the error response
    if 'ResponseMetadata' in e.response:
        print("Request ID:", e.response['ResponseMetadata']['RequestId'])